In [7]:
# %load test.py 
import cv2, dlib
import numpy as np
from imutils import face_utils
from keras.models import load_model
import time
import urllib

In [8]:
def crop_eye(img, eye_points):
  x1, y1 = np.amin(eye_points, axis=0)
  x2, y2 = np.amax(eye_points, axis=0)
  cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

  w = (x2 - x1) * 1.2
  h = w * IMG_SIZE[1] / IMG_SIZE[0]

  margin_x, margin_y = w / 2, h / 2

  min_x, min_y = int(cx - margin_x), int(cy - margin_y)
  max_x, max_y = int(cx + margin_x), int(cy + margin_y)

  eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)

  eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

  return eye_img, eye_rect


In [9]:

url = 'http://172.30.1.29/cam-hi.jpg'
while True:
    imgResp=urllib.request.urlopen(url)
    imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
    img=cv2.imdecode(imgNp,-1)

    # all the opencv processing is done here
    cv2.imshow('test',img)
    if ord('q')==cv2.waitKey(10):
        exit(0)

'''

IMG_SIZE = (34, 26)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

model = load_model('models/2018_12_17_22_58_35.h5')
model.summary()

#현재 눈을 떴는지 감았는지
#1: 눈뜸, 0: 감음
now = 0
#유지한 시간
elapsed = 0
#마지막 뜬 시간
lastOpen = 0
#마지막 감은 시간
lastClosed =0
#최근 다섯번 주기
periods = []
#평균 주기
period = 0

#졸림 시그널
sleepSignal = 0
#시그널 발생 시간
signalTime = 0

# main
#cap = cv2.VideoCapture('http://172.30.1.29/stream')


while True:
    imgResp=urllib.request.urlopen(url)
    imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
    img_ori=cv2.imdecode(imgNp,-1)
  

    img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.5, fy=0.5)

    img = img_ori.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    for face in faces:
        shapes = predictor(gray, face)
        shapes = face_utils.shape_to_np(shapes)

        eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
        eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

        eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
        eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
        eye_img_r = cv2.flip(eye_img_r, flipCode=1)

        cv2.imshow('l', eye_img_l)
        cv2.imshow('r', eye_img_r)

        eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
        eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

        pred_l = model.predict(eye_input_l)
        pred_r = model.predict(eye_input_r)

        if pred_l[0][0]<0.5 :
            # 눈을 감았을 때
            if now==0:
                #감은 상태 유지
                elapsed = time.time()-lastClosed
                if elapsed > 0.5 :
                    #감은상태로 0.5초 이상 경과
                    if signalTime == 0 :
                        #시그널이 가고 있지 않았음
                        sleepSignal = 1
                    elif (time.time()-signalTime)>5:
                        #시그널을 보낸지 5초 이상 경과
                        sleepSignal = 1
            else :
                #감은 상태로 변경
                tmp = time.time()
                now = 0
                elapsed = 0
                lastOpen = tmp

        else:
            #눈을 떳을 때
            if now==1:
                #뜬 상태 유지
                elapsed = time.time()-lastOpen
            else:
                #뜬 상태로 변경
                tmp = time.time()
                periods.append(tmp - lastOpen)
                del periods[0]
                now = 1
                elapsed = 0
                lastClosed = tmp
                period = 0
                for i in periods:
                    period+= i


        if sleepSignal == 1:
            print("졸고있음")
            sleepSignal = 0
            signalTime = time.time()

        # visualize
        state_l = 'O %.1f' if pred_l > 0.1 else '- %.1f'
        state_r = 'O %.1f' if pred_r > 0.1 else '- %.1f'

        state_l = state_l % pred_l
        state_r = state_r % pred_r


        cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
        cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)

        cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
        cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)


        cv2.imshow('result', img)

        if cv2.waitKey(10) == ord('q'):
            break
            
            
'''


URLError: <urlopen error [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다>

In [ ]:
}